In [1]:
from io import BytesIO

import pandas as pd
import numpy as np
import numpy
import panel as pn
pn.extension('tabulator')
import matplotlib.pyplot as plt
import hvplot.pandas
import matplotlib.gridspec as gridspec
import matplotlib
import holoviews as hv
import param
import scipy
from scipy.signal import find_peaks, peak_prominences

In [2]:
cm = 1/2.54  # centimeters in inches

## Color

In [3]:
from itertools import cycle
main_colours = ['yellow', 'navajowhite', 'orange','orangered','red','darkslateblue','mediumpurple','green','yellowgreen','cyan','lightblue']
l_main_colours = cycle(main_colours)
r_main_colours = cycle(main_colours[::-1])
l_sub_colours = cycle(main_colours)
r_sub_colours = cycle(main_colours[::-1])

In [4]:
def get_number_of_rows(dataframe):
    df = pd.read_csv(dataframe)
    return df.shape[0]

In [5]:
def iqr(array):
    q3, q1 = np.percentile(array,[75,25])
    IQR = q3 - q1
    upper_bound = q3 + 1.5*IQR
    lower_bound = q1 - 1.5*IQR
    return lower_bound, upper_bound

In [6]:
@pn.cache
def fetch_data(dataframe_list):
    size = []
    data_frames = []
    try:
        for data in dataframe_list:
                df = pd.read_csv(data)
                df.drop('time', axis = 1, inplace = True)
                df.insert(0,'time',df.index.values)
                df = df.fillna(0)
                size.append(df.shape[0])
                data_frames.append(df)
                minsize = min(size)
    except Exception:
        return None
    return data_frames, minsize


def cut_data(df, maxsize):
    for column in df.keys():
        upper_bound = maxsize
        df = df[df[column] < upper_bound ]
    return df

IndentationError: expected an indented block (2000407357.py, line 6)

## Define widgets

In [ ]:
file_input = pn.widgets.FileInput( sizing_mode='stretch_width',
                                  multiple=True)

In [ ]:
def show_peaks(limbangle, ERROR = 50):
    limbangle = np.asarray(limbangle)
    peaks, _ = scipy.signal.find_peaks(x = limbangle, height = 0, prominence=(4, None))
    prominences = peak_prominences(limbangle, peaks)[0]
    contour_heights = limbangle[peaks] - prominences

    index = []
    i = 0
    for item in peaks:
        if limbangle[item] < ERROR:
            index.append(i)
        i += 1
    peaks = np.delete(peaks, index)
    contour_heights = np.delete(contour_heights,index)
    return peaks, contour_heights


In [ ]:
columns = ['shoulder','elbow','knee','hip']
select_column = pn.widgets.Select(options=columns)
direction = ['l','r']
select_direction =  pn.widgets.Select(options=direction)

# Predict

In [ ]:
import statistics
import numpy as np
import pandas as pd 
def create_features(dataframe, limbangle_raw = 'shoulderLangle'):
    df = pd.read_csv(dataframe)
    df.where(df[limbangle_raw] > 5, inplace = True)
    df.dropna(inplace = True) 
    limbangle = df[limbangle_raw].to_numpy()
    mean_limb = statistics.mean(limbangle)
    median_limb = statistics.median(limbangle)
    variance_limb = statistics.variance(limbangle)
    return_list = [mean_limb,median_limb,variance_limb]

    return return_list

def create_predict_data(dataframe , limbs = ['shoulderRangle', 'shoulderLangle','shoulderLTransv','shoulderRTransv','kneeLangle','kneeRangle','elbowLangle','elbowRangle'] ):
    index = []
    tags = ["mean", "median","variance"]
    row =  []
    df_columns = [tag + limb for limb in limbs for tag in tags ]
    for limb in limbs:
        limb_info = create_features(dataframe = dataframe, limbangle_raw = limb)
        row = row + limb_info
    return pd.DataFrame([row], columns=df_columns)


In [ ]:
import joblib
pca = joblib.load('modelo_pca.joblib')
kmeans = joblib.load('unsupervised-jigsaw_puzzle.joblib')

##HTML Clusters


In [ ]:
cluster_0 = '''
<!DOCTYPE html>
<html lang="en">
<head>
    <meta charset="UTF-8">
    <meta name="viewport" content="width=device-width, initial-scale=1.0">
    <title>Cluster  
 0</title>
    <link rel="stylesheet" href="style.css">
</head>
<body>
    <div class="container">
        <h1>Cluster 0</h1>
        <p>
            Cluster 0 is characterized by a satisfactory range of motion in the upper limbs.
            <br><br>
            This ability to move the arms and shoulders within a normal or near-normal range 
            allows most daily activities to be performed without major difficulties.
            <br><br>
            Preserved range of motion in the upper limbs may be indicative of good joint and muscle health,
            allowing individuals to perform daily tasks.
  
        </p>
    </div>
</body>
</html>
'''

In [ ]:
cluster_1 = '''
<!DOCTYPE html>
<html lang="en">
<head>
    <meta charset="UTF-8">
    <meta name="viewport" content="width=device-width, initial-scale=1.0">
    <title>Cluster  
 0</title>
    <link rel="stylesheet" href="style.css">
</head>
<body>
    <div class="container">
        <h1>Cluster 1</h1>
        <p>

        Cluster 1 presents an intermediate range of motion compared to the other groups.
        While it demonstrates greater restriction than Cluster 2, with its satisfactory range of motion, 
        it maintains a superior movement capacity than Cluster 1, which exhibits significant limitations.
        This moderate range of motion may allow the performance of many daily activities, although with 
        some difficulties or adaptations in tasks that require greater flexibility or joint reach.
  
        </p>
    </div>
</body>
</html>
'''

In [ ]:
cluster_2 = '''
<!DOCTYPE html>
<html lang="en">
<head>
    <meta charset="UTF-8">
    <meta name="viewport" content="width=device-width, initial-scale=1.0">
    <title>Cluster  
 0</title>
    <link rel="stylesheet" href="style.css">
</head>
<body>
    <div class="container">
        <h1>Cluster 2</h1>
        <p>

        Cluster 2 is characterized by very limited range of motion in both the upper and lower limbs.
        This significant restriction in the ability to move joints through their full range can affect 
        the performance of daily activities such as walking, reaching, or getting dressed.'
  
        </p>
    </div>
</body>
</html>
'''

In [ ]:
cluster_None = '''
'''

In [ ]:
def plot(data = None, limb = 'shoulder',direction = 'l', size = (10, 5)):
    plt.subplots_adjust(wspace=0.8, hspace=0.6)
    fig =  plt.figure(num=1, clear=True, figsize=size)
    fig.tight_layout() 
    spec = fig.add_gridspec(ncols=2, nrows=2)
    ax0 = fig.add_subplot(spec[0, :])
    ax1 = fig.add_subplot(spec[1, 0])
    ax2 = fig.add_subplot(spec[1, 1])
    ymax = 1.0
    if file_input.filename != None:    
        dataframes, min_size = fetch_data(file_input.filename)
        for data in dataframes:
            data = cut_data(data, min_size)
            limbLangle = scipy.signal.savgol_filter(data[limb + "L" + "angle"].to_numpy(),51,3)
            limbRangle = scipy.signal.savgol_filter(data[limb + "R" + "angle"].to_numpy(),51,3)
            ax0.legend(loc='upper right', bbox_to_anchor=(1.0, 1.02)) 
            ax1.legend(loc='upper right', bbox_to_anchor=(1.0, 1.02))
            ax2.legend(loc='upper right', bbox_to_anchor=(1.0, 1.02))


            ax0.plot(limbLangle,color = next(l_main_colours))
            ax0.plot(limbRangle, color = next(r_main_colours))
            peaks_l, contour_l = show_peaks(limbLangle, 45)
            peaks_r, contour_r = show_peaks(limbRangle, 45)
            ax0.plot(peaks_l, limbLangle[peaks_l], "x")
            ax0.vlines(x=peaks_l, ymin=contour_l, ymax=limbLangle[peaks_l])
            ax0.plot(peaks_r, limbRangle[peaks_r], "x")
            ax0.vlines(x=peaks_r, ymin=contour_r, ymax=limbRangle[peaks_r])
            ax0.set_title(limb + "L" + "angle & "+limb +"R" + "angle")
            if ax0.get_ylim()[1] > ymax:
                ymax = ax0.get_ylim()[1]
            ax1.set_title(limb + "L" + "angle")
            ax1.plot(limbLangle,color = next(l_sub_colours))
            ax2.set_title(limb + "R" + "angle")
            ax2.plot(limbRangle, color = next(r_sub_colours))
            peaks_l, contour_l = show_peaks(limbLangle, 45)
            peaks_r, contour_r = show_peaks(limbRangle, 45)

            ax1.plot(peaks_l, limbLangle[peaks_l], "x")
            ax1.vlines(x=peaks_l, ymin=contour_l, ymax=limbLangle[peaks_l])
            ax2.plot(peaks_r, limbRangle[peaks_r], "x")
            ax2.vlines(x=peaks_r, ymin=contour_r, ymax=limbRangle[peaks_r])
        fig.set_size_inches(size)
        ax0.set_ylim(0,ymax)
        ax1.set_ylim(0,ymax)
        ax2.set_ylim(0,ymax)
        ax0.legend(loc='upper right', bbox_to_anchor=(1.0, 1.02)) 
        ax1.legend(loc='upper right', bbox_to_anchor=(1.0, 1.02))
        ax2.legend(loc='upper right', bbox_to_anchor=(1.0, 1.02))

        ax0.set(xlabel='frequency (Hz)', ylabel='amplitude (degrees)')
        ax1.set(xlabel='frequency (Hz)', ylabel='amplitude (degrees)')
        ax2.set(xlabel='frequency (Hz)', ylabel='amplitude (degrees)')
    return fig

## Define context binding widgets

In [ ]:
def upload_files(files_ = None):
    if file_input.filename != None:
        file_input.save(file_input.filename)

selected = pn.bind(upload_files,file_input, watch = True)


In [ ]:
selector = pn.widgets.MultiSelect(name=' ', sizing_mode='stretch_width',
options=['Add your files here'])

def update_selector(selector_widget = None, input = None):
    if file_input.filename != None:
        selector.options = file_input.filename
    else:
        selector.options =['Add your files here']
    return selector
selector_widget = pn.bind(update_selector, selector, file_input, watch = True)



In [ ]:
def predict_cluster(file_ = None, selector_ = None):
    cluster = '1000'
    cluster_mapping = {
        '0': cluster_0,
        '1': cluster_1,
        '2': cluster_2,
        '1000': 'cluster_None',
    }
    if selector.value:   
        print(selector.value)
        predict_set = create_predict_data(dataframe="./" + file_input.filename[0]) 
        predict_set['PCA1'], predict_set['PCA2'] = pca.transform(predict_set).T
        predict_set['cluster'] = kmeans.predict(predict_set[['PCA1', 'PCA2']])
        cluster = str(predict_set['cluster'][0])
        message = pn.pane.HTML(cluster_mapping[cluster])
    else:
        message = ''
    return pn.FlexBox(message, sizing_mode="stretch_width",   background="#B0C4DE")

## Debug button

In [ ]:
button = pn.widgets.Button(name='Debug', button_type='primary')
text = pn.widgets.TextInput(value='Ready')

def b(event):
    breakpoint()
def debug(event):
    breakpoint()
button.on_click(b)


## Tab 1 (sidebar & main tab 1)

In [ ]:
exercises = ['e-pong','e-Invaders','e-Puzzle','e-Counter']
selected_exercises = pn.widgets.Select(options = exercises)

In [ ]:
def iqr(array):
    q3, q1 = np.percentile(array,[75,25])
    IQR = q3 - q1
    upper_bound = q3 + 1.5*IQR
    lower_bound = q1 - 1.5*IQR
    return lower_bound, upper_bound

In [ ]:
def boxplot(ywq = None, sdsd = None,direction = 'l'):
    labels = ['left','right']
    fig =  plt.figure(num=1, clear=True, figsize=(8, 4))
    spec = fig.add_gridspec(ncols=2, nrows=2)
    ax0 = fig.add_subplot(spec[0, 0])
    ax1 = fig.add_subplot(spec[0, 1])
    ax2 = fig.add_subplot(spec[1, 0])
    ax3 = fig.add_subplot(spec[1, 1])
    data = selector.value
    limb = ["shoulder","knee","hip","elbow"]
    if not not data: 
        df = fetch_data(data)[0][0]
        ax0.boxplot((df[limb[0] + "L" + "angle"].to_numpy(),df[limb[0] + "R" + "angle"].to_numpy() ), notch=True, vert = True, patch_artist= True, labels=labels)
        ax0.yaxis.grid(True)
        #ax0.set_xlabel('Three separate samples')
        ax0.set_ylabel('Observed values')
        ax0.set_title('Shoulder')

        ax1.boxplot((df[limb[1] + "L" + "angle"].to_numpy(),df[limb[1] + "R" + "angle"].to_numpy() ), notch=True, vert = True, patch_artist= True, labels=labels)
        ax1.yaxis.grid(True)
        #ax1.set_xlabel('Three separate samples')
        ax1.set_ylabel('Observed values')
        ax1.set(xlabel='frequency (Hz)', ylabel='amplitude (degrees)')
        ax1.set_title('Hip')


        ax2.boxplot((df[limb[2] + "L" + "angle"].to_numpy(),df[limb[2] + "R" + "angle"].to_numpy() ), notch=True, vert = True, patch_artist= True, labels=labels)
        ax2.yaxis.grid(True)
        #ax2.set_xlabel('Three separate samples')
        ax2.set_ylabel('Observed values')
        ax2.set_title('Knee')
        ax2.set(xlabel='frequency (Hz)', ylabel='amplitude (degrees)')

        ax3.boxplot((df[limb[3] + "L" + "angle"].to_numpy(),df[limb[3] + "R" + "angle"].to_numpy() ), notch=True, vert = True, patch_artist= True, labels=labels)
        ax3.yaxis.grid(True)
        #ax3.set_xlabel('Three separate samples')
        ax3.set_ylabel('Observed values')
        ax3.set_title('Elbow')
        ax3.set(xlabel='frequency (Hz)', ylabel='amplitude (degrees)')
    return fig

## Tab 2

In [ ]:
def plot_details(data = None, limb = 'shoulder', direction = 'l', size = (5.5, 3)):
    fig =   plt.figure(num=1, clear=True, figsize=size)
    spec = fig.add_gridspec(ncols=1, nrows=1)
    ax = fig.add_subplot(spec[0, :])
    if not not selector.value:
        data, min_size = fetch_data(selector.value)
        [data] = data
        limbLangle = scipy.signal.savgol_filter(data[limb + "L" + "angle"].to_numpy(),51,3)
        limbRangle = scipy.signal.savgol_filter(data[limb + "R" + "angle"].to_numpy(),51,3)
        ax.plot(limbLangle,color = next(l_main_colours))
        ax.plot(limbRangle, color = next(r_main_colours))
        peaks_l, contour_l = show_peaks(limbLangle, 45)
        peaks_r, contour_r = show_peaks(limbRangle, 45) 

        ax.plot(peaks_l, limbLangle[peaks_l], "x")
        ax.vlines(x=peaks_l, ymin=contour_l, ymax=limbLangle[peaks_l])
        ax.plot(peaks_r, limbRangle[peaks_r], "x")
        ax.vlines(x=peaks_r, ymin=contour_r, ymax=limbRangle[peaks_r])
        ax.set_title(limb + "L" + "angle & "+limb +"R" + "angle") 
        ax.set(xlabel='frequency (Hz)', ylabel='amplitude (degrees)')
    return fig



In [ ]:
from PIL import Image
fig = Image.open('./clusters.png')
clusters = pn.pane.Image(fig)


In [ ]:

details_widget = pn.bind(plot_details,selector, select_column, selector_widget,watch= True)
boxplot_widget = pn.bind(boxplot,selector,selector_widget,watch= True)
main_widget = pn.bind(plot,file_input,select_column,select_direction, watch = True)
selected_cluster = pn.bind(predict_cluster,selector,watch= True)

## Documentation

In [ ]:
doc_markdown = ''' # Clusters Description 

## Cluster 0 

Cluster 0 is characterized by a satisfactory range of motion in the upper limbs. 
This ability to move the arms and shoulders within a normal or near-normal range 
allows most daily activities to be performed without major difficulties.
Preserved range of motion in the upper limbs may be indicative of good joint and muscle health, 
allowing individuals to perform daily tasks.


## Cluster 1 

Cluster 1 presents an intermediate range of motion compared to the other groups.
While it demonstrates greater restriction than Cluster 2, with its satisfactory range of motion, 
it maintains a superior movement capacity than Cluster 1, which exhibits significant limitations.
This moderate range of motion may allow the performance of many daily activities, although with 
some difficulties or adaptations in tasks that require greater flexibility or joint reach.


## Cluster 2

Cluster 2 is characterized by very limited range of motion in both the upper and lower limbs.
 This significant restriction in the ability to move joints through their full range can affect 
 the performance of daily activities such as walking, reaching, or getting dressed.'''

In [ ]:
markdown = pn.pane.Markdown(doc_markdown)

## Create the Template

In [ ]:
details_grid = pn.GridSpec(ncols = 2, nrows = 2)
details_grid[0, 0] = pn.Column(details_widget,  sizing_mode='stretch_width')
details_grid[0, 1] = pn.Column(boxplot_widget, sizing_mode='stretch_width')
details_grid[1, 0] = selected_cluster
details_grid[1, 1] = None

In [ ]:
#Layout using Template
template = pn.template.BootstrapTemplate(
    title='Data analysis for motor disabilities rehabilitation of the upper limbs using augmented reality', 
    sidebar=[pn.pane.Markdown(""), 
             file_input,
             button,
             select_column,
             select_direction,
             pn.pane.Markdown("### Selected Files"), 
             selector_widget
            ],
    main=[ pn.Tabs(("Main", main_widget),("Details", details_grid), ("Documentation", pn.Column(markdown,clusters)))],
    header_background='purple',
)
template.show()
template.servable();